In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.sparse import csr_matrix, csc_matrix, find, coo_matrix
from scipy.sparse import eye, spdiags
from scipy.sparse.linalg import spsolve
from scipy.linalg import eigh
from scipy.io import  savemat


Your Python environment should include `numpy`, `scipy`, `matplotlib` and `meshplot`. The latter can be installed from conda using:
><code> conda install -c conda-forge meshplot
</code>

In [2]:
import sys
sys.path.append("./utils")
import mesh_utils as plu # Follow the above procedure
import wasserstein_utils as wass
import diffusion_utils as diff
from IPython.display import display, HTML

# Implementation of 3D meshes class

## Read OFF files

In [3]:
def read_off(filepath):
    """
    Reads a simple .off file
    
    Input
    --------------
    filepath : str - path to the .off file
    
    Output
    --------------
    vertices : (n,3) array of vertex coordinates (float)
    faces    : (m,3) array of faces defined by vertices index (integers)
    """
    
    with open(filepath, 'r') as f:
        f.readline() # First line
        n_vertices, n_triangles, _ = f.readline().strip().split() # nb of vertices triangles and 0
        
        vertices, faces = [], []

        for i in range(int(n_vertices)):
            x, y, z = f.readline().strip().split()
            vertices.append([float(x), float(y), float(z)])

        for j in range(int(n_triangles)):
            _, i, j , k = f.readline().strip().split()
            faces.append([int(i), int(j), int(k)])
    
    return np.array(vertices), np.array(faces)

## Mesh class

In [4]:
class Mesh3D:
    def __init__(self, path):
        """
        Initialize the mesh from a path
        """
        self.vertices, self.faces = read_off(path)
        self.A = None
        self.W = None


    def compute_faces_areas(self):
        """
        Compute the area of each face
        
        Input
        --------------
        vertices : (n,3) - vertex coordinates
        faces    : (m,3) - faces defined by vertex indices
        
        Output
        --------------
        faces_areas : (m,) - area of each face
        """
        v1 = self.vertices[self.faces[:,0]]  # (m,3)
        v2 = self.vertices[self.faces[:,1]]  # (m,3)
        v3 = self.vertices[self.faces[:,2]]  # (m,3)
        faces_areas = 0.5 * np.linalg.norm(np.cross(v2-v1,v3-v1),axis=1)  # (m,)

        return faces_areas
    
    def area_matrix(self):
        """
        Compute the diagonal area matrix
        
        Input
        --------------
        vertices : (n,3) - vertex coordinates
        faces    : (m,3) - faces defined by vertex indices
        
        Output
        --------------
        A : (n,n) sparse matrix in DIAgonal format
        """

        N = self.vertices.shape[0]

        # Compute face area

        v1 = self.vertices[self.faces[:,0]]  # (m,3)
        v2 = self.vertices[self.faces[:,1]]  # (m,3)
        v3 = self.vertices[self.faces[:,2]]  # (m,3)
        faces_areas = 0.5 * np.linalg.norm(np.cross(v2-v1,v3-v1),axis=1)  # (m,)

        I = np.concatenate([self.faces[:,0], self.faces[:,1], self.faces[:,2]])
        J = np.zeros_like(I)
        V = np.concatenate([faces_areas, faces_areas, faces_areas])/3

        # Get array of vertex areas
        vertex_areas = np.array(scipy.sparse.coo_matrix((V, (I, J)), shape=(N, 1)).todense()).flatten()

        A = scipy.sparse.dia_matrix((vertex_areas,0), shape=(N, N))
        self.A = A
        return A
    
    def cotan_matrix(self):
        """
        Compute the stiffness matrix
        
        Input
        --------------
        vertices : (n,3) - vertex coordinates
        faces    : (m,3) - faces defined by vertex indices
        
        Output
        --------------
        W : (n,n) sparse matrix in CSC format
        """
        N = self.vertices.shape[0]

        v1 = self.vertices[self.faces[:,0]]  # (m,3)
        v2 = self.vertices[self.faces[:,1]]  # (m,3)
        v3 = self.vertices[self.faces[:,2]]  # (m,3)

        # Edge lengths indexed by opposite vertex
        u1 = v3 - v2
        u2 = v1 - v3
        u3 = v2 - v1

        L1 = np.linalg.norm(u1,axis=1)  # (m,)
        L2 = np.linalg.norm(u2,axis=1)  # (m,)
        L3 = np.linalg.norm(u3,axis=1)  # (m,)

        # Compute cosine of angles
        A1 = np.einsum('ij,ij->i', -u2, u3) / (L2*L3)  # (m,)
        A2 = np.einsum('ij,ij->i', u1, -u3) / (L1*L3)  # (m,)
        A3 = np.einsum('ij,ij->i', -u1, u2) / (L1*L2)  # (m,)

        # Use cot(arccos(x)) = x/sqrt(1-x^2)
        I = np.concatenate([self.faces[:,0], self.faces[:,1], self.faces[:,2]])
        J = np.concatenate([self.faces[:,1], self.faces[:,2], self.faces[:,0]])
        S = np.concatenate([A3,A1,A2])
        S = 0.5 * S / np.sqrt(1-S**2)

        In = np.concatenate([I, J, I, J])
        Jn = np.concatenate([J, I, I, J])
        Sn = np.concatenate([-S, -S, S, S])

        W = scipy.sparse.coo_matrix((Sn, (In, Jn)), shape=(N, N)).tocsc()
        self.W = W
        return W
    
    def compute_laplacian(self):
        self.A = self.area_matrix()
        self.W = self.cotan_matrix()

    def compute_eigendecomposition(self, K:int):
        if self.W is None or self.A is None:
            self.compute_laplacian()
        self.eigenvalues, self.eigenvectors = scipy.sparse.linalg.eigsh(self.W, M=self.A,
                                                                  k=K, sigma=-0.01)
        
    def compute_heatkernel(self, eigenvalues, eigenvectors):
        pass
        #TODO use of lambdify to compute only once a Callable


In [5]:
def diffuse(f, mesh:Mesh3D, t):
    """
    Diffuse a function f on a mesh for time t
    
    Input
    --------------
    f       : (n,) - function values
    mesh    : MyMesh - mesh on which to diffuse
    t       : float - time for which to diffuse
    
    Output
    --------------
    f_diffuse : (n,) values of f after diffusion
    """


    mesh.compute_laplacian()
    
    f_diffuse = scipy.sparse.linalg.spsolve(mesh.A + t*mesh.W, mesh.A@f)
    
    return f_diffuse

### Example

In [6]:
mesh = Mesh3D("../data/nefertiti.off")

### Visualizing the Eigenfunctions of the Laplacian

In [7]:
p = plu.plot(mesh)
# html = p.to_html(imports=True, html_frame=False)
# display(HTML(html))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0293350…

In [8]:
mesh.compute_eigendecomposition(K=100)

In [9]:
first_eigenfunction = mesh.eigenvalues[0]*mesh.eigenvectors[:, 0]
plu.plot(mesh, first_eigenfunction, colormap = "coolwarm")

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0293350…

In [10]:
last_eigenfunction = mesh.eigenvalues[-1]*mesh.eigenvectors[:, -1]
plu.plot(mesh, last_eigenfunction, colormap = "coolwarm")

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0293350…

## Diffusion of a Dirac function

In [12]:
# Dirac on a random vertex

dirac = np.array([0]*(mesh.vertices.shape[0]))
dirac[20]= 1
dirac[150]= 1
dirac_diffused = diff.diffuse_mesh(dirac, mesh, 0.1)

plu.plot(mesh, dirac_diffused, colormap="coolwarm")

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0293350…

## Barycenter

In [34]:
entropy = 5

f_heatkernel = lambda x: diff.diffuse_mesh(x, mesh, entropy/2)
dirac1 = np.array([0]*(mesh.vertices.shape[0]))
dirac2 = np.array([0]*(mesh.vertices.shape[0]))
dirac1[20]= 1
dirac2[150]= 1
a = csr_matrix.diagonal(mesh.area_matrix().tocsr())

for t in range(0,11):
    barycenter_mu, error = wass.wasserstein_barycenter(np.array([dirac1,dirac2]).T, a, f_heatkernel,lambd=[t/10,1-t/10],n_iter=500)
    plu.plot(mesh, barycenter_mu, colormap="coolwarm")

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0293350…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0293350…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0293350…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0293350…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0293350…

...

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0293350…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0293350…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0293350…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0293350…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0293350…